In [ ]:
import pandas as pd
import json
from collections import defaultdict


def load_data(file_path):
    """Load a CSV into a DataFrame."""
    return pd.read_csv(file_path)


def load_all_data(filepaths):
    """Load multiple CSVs and return a list of DataFrames."""
    return [load_data(fp) for fp in filepaths]


def merge_dataframes(df_list, how='outer'):
    """Merge a list of DataFrames pairwise using the specified method."""
    from functools import reduce
    return reduce(lambda left, right: pd.merge(left, right, how=how), df_list)


def remove_collumn(df, string='raw'):
    """Drop any column with string in its name."""
    cols_to_drop = [c for c in df.columns if string in c]
    return df.drop(columns=cols_to_drop) if cols_to_drop else df


def expand_list_column(df, column_name, prefix, rename_map=None):
    """
    Expand a column of lists or JSON-encoded lists into separate numeric columns.

    Parameters:
        df: DataFrame containing the list column.
        column_name: Name of column to expand.
        prefix: Prefix for new columns.
        rename_map: Optional dict mapping default names to custom names.

    Returns:
        DataFrame with expanded, numeric columns.
    """
    def to_list(x):
        if isinstance(x, str):
            try:
                return json.loads(x)
            except json.JSONDecodeError:
                return []
        return x if isinstance(x, (list, tuple)) else []

    series_lists = df[column_name].apply(to_list)
    expanded = pd.DataFrame(series_lists.tolist(), index=df.index)
    default_names = [f"{prefix}_{i}" for i in range(expanded.shape[1])]
    expanded.columns = default_names

    # Cast expanded columns to numeric type
    expanded = expanded.apply(pd.to_numeric, errors='coerce')

    if rename_map:
        expanded.rename(columns=rename_map, inplace=True)

    return pd.concat([df.drop(columns=[column_name]), expanded], axis=1)


def rename_score_column(df, old_name, new_name):
    """Rename a single column."""
    return df.rename(columns={old_name: new_name})


def apply_score_mapping(df, mapping, exclude_cols=None):
    """
    Map numeric codes to labels for all numeric columns except those excluded.

    Parameters:
        df: DataFrame to process.
        mapping: dict of {code: label}.
        exclude_cols: list of column names to skip.

    Returns:
        DataFrame with values mapped where applicable.
    """
    exclude = set(exclude_cols or [])
    for col in df.columns:
        if col not in exclude and pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].map(mapping).fillna(df[col])
    return df


def process_all_scores(df_scores, merged_df, prefix='anno', rename_map=None):
    """Extract, expand as numeric, and merge the all_scores list column."""
    subset = df_scores[['id', 'all_scores']].copy()
    expanded = expand_list_column(subset, 'all_scores', prefix, rename_map)
    return pd.merge(merged_df, expanded, on='id', how='left')


def get_column_averages(df, exclude_cols=None):
    """Return a dict of mean value for each numeric column, excluding given columns."""
    exclude = set(exclude_cols or [])
    averages = {}
    for col in df.columns:
        if col not in exclude and pd.api.types.is_numeric_dtype(df[col]):
            averages[col] = df[col].mean()
    return averages


def save_to_json(df, file_path):
    """Save DataFrame to a JSON file (records, lines)."""
    df.to_json(file_path, orient='records', lines=True)
    print(f"Data saved to {file_path}")


def main():
    print('main() started')
    # File paths
    paths = [
        'test_results_with_predictions.csv',
        '../eval/gemini_predictions.csv',
        '../self_annotation/test_final.csv'
    ]

    # Load data
    df1, df2, df3 = load_all_data(paths)

    display(df3.head())

    # Rename and merge initial DataFrames
    df2 = rename_score_column(df2, 'int_score', 'gemini-2.5-flash_score')
    merged = merge_dataframes([df1, df2])
    merged = merged.drop(columns=['real_label'], errors='ignore')
    merged = remove_collumn(merged, 'int_score')

    # Expand annotations from all_scores
    rename_map = {
        'anno_0': 'own_anno_1',
        'anno_1': 'own_anno_2',
        'anno_2': 'own_anno_3',
        'anno_3': 'anno_1',
        'anno_4': 'anno_2',
        'anno_5': 'anno_3'
    }
    merged = process_all_scores(df3, merged, prefix='anno', rename_map=rename_map)

    # Get averages dict and print
    averages = get_column_averages(merged, exclude_cols=['id', 'text'])
    for col, mean_val in averages.items():
        print(f"{col}: {mean_val:.2f}")

    # Display head and save final JSON
    display(merged.head())
    save_to_json(merged, 'final_results.json')

if __name__ == '__main__':
    main()


main() started


,id,text,educational_value_labels,annotator_ids,problematic_content_label_present,problematic_content_label_agreement,language_names,language_code,all_labels,all_scores,int_score
0,d4ffcabc-e77a-48bb-8bf4-735683199642,Jobannonce: Chemist\nOBS: annoncen er udløbet!...,"['None', 'None', 'None']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '85a...,False,1.0,dan_Latn,dan_Latn,"['None', 'None', 'None', 'None', 'None']","[0, 0, 0, 0, 0]",0
1,fbdbb330-dd3a-4a66-925b-01a1bbfe6ac5,Det europæiske år for færdigheder: Kommissione...,"['Basic', 'Basic', 'Minimal']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '85a...,False,1.0,dan_Latn,dan_Latn,"['Basic', 'Basic', 'Minimal', 'Minimal', 'Good']","[2, 2, 1, 1, 3]",2
2,c6e11cad-e753-4da8-a9ab-72f0e06ea398,"Det ligner ikke, at Wan-Bissaka har en fremtid...","['None', 'None', 'Basic']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '85a...,False,1.0,dan_Latn,dan_Latn,"['None', 'None', 'Basic', 'Minimal', 'None']","[0, 0, 2, 1, 0]",1
3,09a4bdba-5b5f-49b8-9530-a424ab6868a5,"Sauro Maule 2017 ""Pri"" Garganega\nAromatisk or...","['Basic', 'None', 'None']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '740...,False,1.0,dan_Latn,dan_Latn,"['Basic', 'None', 'None', 'Minimal', 'None']","[2, 0, 0, 1, 0]",1
4,d4e44cf0-1147-46ce-9280-e82d2589a5e3,fyens.dk bruger cookies til at give dig en god...,"['None', 'None', 'None']",['a0585a5c-b72f-4c3a-a2a3-17e8e0b4ea4f'\n '998...,False,1.0,dan_Latn,dan_Latn,"['None', 'None', 'None', 'None', 'None']","[0, 0, 0, 0, 0]",0


raw_prediction_zeroshot: 1.37
raw_prediction_Full-finetune: 1.25
raw_prediction_fewshot-250-samples: 1.51
raw_prediction_fewshot-1000-samples: 1.39
raw_prediction_fewshot-2500-samples: 1.44
gemini-2.5-flash_score: 0.88
own_anno_1: 0.84
own_anno_2: 0.83
own_anno_3: 1.18
anno_1: 1.34
anno_2: 0.99


,id,text,raw_prediction_zeroshot,raw_prediction_Full-finetune,raw_prediction_fewshot-250-samples,raw_prediction_fewshot-1000-samples,raw_prediction_fewshot-2500-samples,gemini-2.5-flash_score,own_anno_1,own_anno_2,own_anno_3,anno_1,anno_2
0,f747cbcd-2ced-4d5a-ad8e-b57b1f6151da,"""Det ganske lille kongerige var virkelig meget...",1.659694,1.635776,1.794578,1.679552,1.736339,3,1,0,0,1,0
1,ecde814d-1ac3-46c6-a851-d5f99ea8445a,#5 Nu er det så smart med Valve at de har stea...,1.073710,0.759104,1.204092,1.089823,1.123433,0,0,1,0,0,1
2,17bddd3a-73f8-4df3-a626-86772f6353b4,"(2. sektion)\nNedslag\nNORDISK TIDSKRIFT, som ...",1.805018,1.952327,1.943254,1.828237,1.887721,1,0,1,3,2,2
3,afbb9f1d-65b3-4eb7-85fa-52310160a511,"- Den største del, nemlig 40 procent, forsvind...",1.675730,1.695088,1.813372,1.695861,1.748217,3,1,1,1,1,1
4,908e22f5-484c-4ddd-a975-85a56229da2f,22.02.2017 | Medarbejder\nRapport: Tekstilprod...,1.339737,1.226509,1.473391,1.358661,1.405062,1,1,3,0,2,1


Data saved to final_results.json
